In [ ]:
New method, not working yet


def HROI2(frame2frame_changes, min_area=300):
    
    aft_frame2frame_changes = []
    pixels_freq = []   
    frame2frame_changes_filtering = assert_8bit(frame2frame_changes)
    changes_video_filtering = normVideo(frame2frame_changes_filtering)
    
    for idx, dis_points in enumerate(changes_video_filtering):
        contours_f, _ = cv2.findContours(dis_points, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        biggest_cnt = []
        for cnt in contours_f:
            area_cnt = cv2.contourArea(cnt)
            biggest_cnt.append(area_cnt)
          
            max_value = max(biggest_cnt)
            max_index = biggest_cnt.index(max_value)
            zero_mask = np.zeros_like(dis_points)
            zero_maks_3ch = cv2.cvtColor(zero_mask,cv2.COLOR_GRAY2RGB)
           

        area_cnt_final = cv2.contourArea(contours_f[max_index])
        area_cnt_final = int(area_cnt_final/1000*255)   #Put the big numbers resulting from arrays sum into a reasonable number betwen 0 and 255
        if area_cnt_final > 255:
            area_cnt_final = 0
                    
        cv2.drawContours(zero_maks_3ch, [contours_f[max_index]], -1, (area_cnt_final, area_cnt_final, area_cnt_final), -1)
        
      
        mask_count = cv2.cvtColor(zero_maks_3ch,cv2.COLOR_RGB2GRAY)      


        mask_count[mask_count > 0] = 1       
        pixels_freq.append(mask_count)       
        
          
       
        
        zero_maks_1ch = cv2.cvtColor(zero_maks_3ch,cv2.COLOR_RGB2GRAY)
        aft_frame2frame_changes.append(zero_maks_1ch)
        
    
    #total_frames = len(bin_mask)
    #############################################################################################################################
    temp_array = np.zeros((900, 900))
    temp_array = temp_array.flatten() 
    
      
    
    max_array = np.zeros((900, 900))
    max_array = max_array.flatten()
    
    
        
    for frame_f in pixels_freq:  
        
      
        frame_f = frame_f.flatten()
        
      
               
        for idx, n in enumerate(frame_f):
          
            if n == 0:
                
                temp_array[idx] = temp_array[idx] + 1
               
                if temp_array[idx] > max_array[idx]:
                    max_array[idx] = temp_array[idx]
                    
            else:
                temp_array[idx] = 0
                        
    print("min")
    print(np.amin(max_array))
    
    print("max")
    print(np.amax(max_array))
    
    max_array.reshape(900,900)
    
    
        
    
    #total_changes_binary = np.sum(max_array, axis=0, dtype=np.uint8)    # what is it?
    
    print(max_array)
    
    cv2.imshow("img", max_array)
    cv2.waitKey(0)
                                                                     
                                                                   
                                                                       
    # TODO: a lot of resolution is lost with the conversion. to uint16
    
    threshold = 50 #int(len(total_changes_binary)*0.5)
    
    bool_mask = total_changes_binary < threshold
    
    
    final_masked = []
    for frame in aft_frame2frame_changes:
        
        
        frame[np.where(bool_mask)] = 0
        final_masked.append(frame)
    
    
   
    
    
    total_changes = np.sum(final_masked, axis=0, dtype=np.uint8) # will be used to identify the ROI based on a formula of frequency and intensity of the pixel
    
    
    
    ### Create mask with most changing pixels
    nr_of_pixels_considered = 250
    top_changing_indices = np.unravel_index(np.argsort(total_changes.ravel())[-nr_of_pixels_considered:], total_changes.shape)
    
    top_changing_mask = np.zeros((total_changes.shape), dtype=bool)

    # Label pixels based on the top changeable pixels
    top_changing_mask[top_changing_indices] = 1

    ### Threshold heart RoI to find regions
    all_roi = total_changes > threshold_yen(total_changes)
    all_roi = all_roi.astype(np.uint8)
    
    #cv2.imshow("img_y", all_roi*255)
    #cv2.waitKey(0)
    
    
   

    # Fill holes in blobs
    KERNEL = np.ones((9, 9), np.uint8)

    all_roi = cv2.morphologyEx(all_roi, cv2.MORPH_CLOSE, KERNEL)
    
    #cv2.imshow("img_y2", all_roi*255)
    #cv2.waitKey(0)

  

    hroi_mask = hroi_from_blobs(all_roi, top_changing_mask)
    
        
   
    
    
    
    return hroi_mask, all_roi, total_changes, top_changing_mask   # all_roi segundo